<a href="https://colab.research.google.com/github/Mfys212/GNN-for-Molekul-Classifier/blob/main/GNN_Molekul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric
!pip install rdkit

In [ ]:
import pandas as pd
import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.data import DataLoader as DL_geo
from torch_geometric.nn import GCNConv, global_mean_pool, GINEConv
from torch.utils.data import DataLoader, TensorDataset
from rdkit import Chem
from rdkit.Chem import rdmolops, AllChem
import torch.nn.functional as F
import random
from imblearn.over_sampling import RandomOverSampler

In [ ]:
df_2class_prep = pd.read_csv('https://raw.githubusercontent.com/Mfys212/dokumen/refs/heads/main/01_label2class_sars.csv')
df_2class_prep

,molecule_chembl_id,class,canonical_smiles,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL541133,active,Cc1ccc([N+](=O)[O-])cc1S(=O)(=O)c1nc(-c2cnc3cc...,434.886,4.16062,0.0,8.0,7.000000
1,CHEMBL538346,active,Oc1cccc(-c2nc(N3CCOCC3)c3oc4ncccc4c3n2)c1,348.362,2.98020,1.0,7.0,8.000000
2,CHEMBL541643,active,Oc1cccc(-c2nc(N3CCOCC3)c3sccc3n2)c1,313.382,2.90050,1.0,6.0,6.657577
3,CHEMBL556399,active,Cc1ccc([N+](=O)[O-])cc1S(=O)(=O)N(C)/N=C/c1cnc...,452.290,2.96802,0.0,7.0,7.000000
4,CHEMBL538346,active,Oc1cccc(-c2nc(N3CCOCC3)c3oc4ncccc4c3n2)c1,348.362,2.98020,1.0,7.0,8.000000
...,...,...,...,...,...,...,...,...
5805,CHEMBL5279152,active,CCn1c(=O)c2ccnn2c2cc(N)c(C(=O)NCc3cccnc3)cc21,362.393,1.57630,2.0,7.0,6.060481
5806,CHEMBL5269677,active,CCn1c(=O)c2ccnn2c2cc(N)c(C(=O)NCc3cncnc3)cc21,363.381,0.97130,2.0,8.0,6.619789
5807,CHEMBL5279774,inactive,CCn1c(=O)c2ccnn2c2cc(NC)c(C(=O)NCc3cccnc3)cc21,376.420,2.03580,2.0,7.0,4.698970
5808,CHEMBL5271447,inactive,CCn1c(=O)c2ccnn2c2cc(N(C)C)c(C(=O)NCc3cccnc3)cc21,390.447,2.06010,1.0,7.0,4.698970


In [ ]:
data = df_2class_prep[["canonical_smiles", "class"]]
print(data.isnull().sum(), "\n")
data.info()

canonical_smiles    0
class               0
dtype: int64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5810 entries, 0 to 5809
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   canonical_smiles  5810 non-null   object
 1   class             5810 non-null   object
dtypes: object(2)
memory usage: 90.9+ KB


In [ ]:
class_counts = data["class"].value_counts()
class_counts

,count
class,
active,4846
inactive,964


In [ ]:
def generate_conformations(smiles, num_conformations=5):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return []
    mol = Chem.AddHs(mol)
    AllChem.EmbedMultipleConfs(mol, numConfs=num_conformations)
    conformations = []
    for conf in range(num_conformations):
        conformations.append(Chem.MolToSmiles(mol, isomericSmiles=True))
    return conformations

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(data[['canonical_smiles']], data['class'])

oversampled_data = []
for smiles, cls in zip(X_resampled['canonical_smiles'], y_resampled):
    conformations = generate_conformations(smiles)
    for conf in conformations:
        oversampled_data.append({'canonical_smiles': conf, 'class': cls})

balanced_data = pd.DataFrame(oversampled_data)

[12:02:21] UFFTYPER: Unrecognized charge state for atom: 8
[12:02:21] UFFTYPER: Unrecognized charge state for atom: 8
[12:03:07] UFFTYPER: Unrecognized charge state for atom: 22
[12:07:27] UFFTYPER: Unrecognized charge state for atom: 9
[12:07:38] UFFTYPER: Unrecognized charge state for atom: 10
[12:09:37] UFFTYPER: Unrecognized charge state for atom: 16
[12:09:52] UFFTYPER: Unrecognized charge state for atom: 16
[12:10:30] UFFTYPER: Unrecognized charge state for atom: 16
[12:11:19] UFFTYPER: Unrecognized charge state for atom: 16


In [ ]:
print(balanced_data.info(), "\n")
balanced_data.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48460 entries, 0 to 48459
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   canonical_smiles  48460 non-null  object
 1   class             48460 non-null  object
dtypes: object(2)
memory usage: 757.3+ KB
None 



,,count
canonical_smiles,class,
[H]Oc1c([H])c([H])c([H])c(-c2nc(N3C([H])([H])C([H])([H])OC([H])([H])C3([H])[H])c3oc4nc([H])c([H])c([H])c4c3n2)c1[H],active,180
[H]c1c([H])c([H])c2c(c([H])nn2[H])c1-c1nc(N2C([H])([H])C([H])([H])OC([H])([H])C2([H])[H])c2sc(C([H])([H])N3C([H])([H])C([H])([H])N(S(=O)(=O)C([H])([H])[H])C([H])([H])C3([H])[H])c([H])c2n1,active,160
[H]c1nc([H])c(C([H])([H])N2c3nc(N4C([H])([H])C([H])([H])OC([H])([H])[C@@]4([H])C([H])([H])[H])c([H])c(=O)n3C([H])([H])C([H])([H])[C@@]2([H])C(F)(F)F)c([H])c1Cl,inactive,150
[H]c1nc2c([H])c([H])c([H])c([H])c2c([H])c1-c1c([H])c([H])c2nc([H])c3c(c2c1[H])n(-c1c([H])c([H])c(C(C#N)(C([H])([H])[H])C([H])([H])[H])c([H])c1[H])c(=O)n3C([H])([H])[H],active,130
[H]c1nc(N([H])[H])c2nc([H])n(C([H])([H])c3nc4c([H])c([H])c([H])c(C([H])([H])[H])c4c(=O)n3-c3c([H])c([H])c([H])c([H])c3C([H])([H])[H])c2n1,inactive,100
...,...,...
[H]c1c([H])c([H])c(-n2nc([H])c3c(N4C([H])([H])C([H])([H])OC([H])([H])C4([H])[H])nc(-c4c([H])c([H])c(N([H])C(=O)C([H])([H])[H])c([H])c4[H])nc32)c([H])c1[H],active,5
[H]c1c([H])c([H])c(-n2nc([H])c3c(N4C([H])([H])C([H])([H])OC([H])([H])C4([H])[H])nc(-c4c([H])c([H])c(N([H])C(=O)OC([H])([H])[H])c([H])c4[H])nc32)c([H])c1[H],active,5
[H]c1c([H])c([H])c(-n2nc([H])c3c(N4C([H])([H])C([H])([H])OC([H])([H])C4([H])[H])nc(-c4c([H])c([H])c(N([H])[H])c([H])c4[H])nc32)c([H])c1[H],active,5
